# 📓 The GenAI Revolution Cookbook

**Title:** How to Use GPT Prompts for Test Case Generation in QA [2025]

**Description:** Generate comprehensive, reliable tests in minutes using proven GPT prompts, BDD templates, and tool integrations—cut flakiness, expand coverage, accelerate releases.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Generating test scenarios from acceptance criteria often produces shallow, happy-path-only outputs that ignore edge cases, security risks, and format requirements. This happens because instruction dilution and missing delimiters cause format contamination and incomplete coverage.

This guide shows you a five-part prompt scaffold to generate schema-compliant Gherkin test suites from acceptance criteria. You'll see a before/after comparison where a naive prompt yields 3 generic scenarios, and a structured prompt yields 12 schema-valid scenarios with boundary and OWASP coverage.

## What Problem Are We Solving?

When you prompt a language model to generate test scenarios, you typically get:

- Only happy-path cases with no boundary or negative tests.
- Outputs that ignore acceptance criteria or blend them into prose.
- Gherkin syntax errors that break parsers and runners.
- Missing security or edge-case coverage.

For example, a naive prompt like "Generate test scenarios for this user story" produces 3 generic scenarios focused on success flows. A structured prompt with delimiters, explicit counts, and schema constraints produces 12 parseable scenarios covering happy paths, boundaries, negatives, and OWASP risks.

## What's Actually Happening Under the Hood?

Four mechanisms explain why unstructured prompts fail:

- **Recency and salience bias.** Models prioritize instructions near the generation task. If acceptance criteria appear early and the generation instruction appears late, the model may ignore the criteria.
- **Instruction dilution.** Mixing requirements, format rules, and constraints in a single block reduces compliance. The model treats everything as suggestions rather than hard rules.
- **Format contamination.** Without delimiters, the model blends instructions into the output. You get explanatory text instead of clean Gherkin.
- **Exploration vs. compliance tension.** Models default to creative, varied outputs. Without explicit constraints (counts, tags, schema fields), they skip edge cases and security scenarios.

The following diagram shows how proper ordering and delimiters prevent contamination:

```mermaid
flowchart LR
    A[System: Role + Style] --> B[Context: User Story + Criteria]
    B --> C[Task: Coverage + Counts]
    C --> D[Format: Gherkin Schema]
    D --> E[Constraints: Negative + OWASP + Boundaries]
    E --> F[Output: Tests]
    B -. no delimiters .-> F
```

## How to Fix It: Structured Gherkin Outputs with Delimiters and Schema Constraints

Use a five-part scaffold to control output quality:

1. **Role (system message).** Assign a QA engineer persona with strict terminology.
2. **Context (user message).** Provide the user story and acceptance criteria, delimited with `---CRITERIA---` and `---END CRITERIA---`.
3. **Task.** Specify coverage targets and counts (e.g., "Generate 12 scenarios: 4 happy, 4 boundary, 2 negative, 2 OWASP").
4. **Format.** Define the Gherkin schema with required fields (Feature, Scenario, Given/When/Then, tags).
5. **Constraints.** Enforce output-only blocks, no explanations, and explicit tagging (e.g., `@security @owasp-a01`).

### Before: Naive Prompt

In [ ]:
Generate test scenarios for a login feature.

**Output (3 scenarios, all happy paths):**

```gherkin
Scenario: User logs in successfully
  Given the user is on the login page
  When the user enters valid credentials
  Then the user is redirected to the dashboard
```

### After: Structured Prompt

This example demonstrates the five-part scaffold with delimiters, explicit counts, and schema enforcement.

In [ ]:
System: You are a QA engineer. Use strict Gherkin syntax. No explanations.

User:
---CRITERIA---
AC-1: Users must log in with email and password.
AC-2: Invalid credentials show an error.
AC-3: Account locks after 5 failed attempts.
---END CRITERIA---

Generate 12 scenarios:
- 4 happy paths
- 4 boundary cases (empty fields, special characters, max length)
- 2 negative cases (wrong password, locked account)
- 2 OWASP cases (SQL injection, brute force)

Format:
Feature: [name]
Scenario: [description]
  Given [precondition]
  When [action]
  Then [outcome]
Tags: @type @owasp-id (if applicable)

Output only Gherkin. No commentary.

**Output (12 scenarios, schema-compliant, with boundary and security coverage):**

```gherkin
Feature: User Login

@happy
Scenario: Valid login redirects to dashboard
  Given the user is on the login page
  When the user enters valid email and password
  Then the user is redirected to the dashboard

@boundary
Scenario: Empty email field shows validation error
  Given the user is on the login page
  When the user submits with an empty email field
  Then an error message "Email is required" is displayed

@negative
Scenario: Locked account prevents login
  Given the user account is locked after 5 failed attempts
  When the user enters valid credentials
  Then an error message "Account locked" is displayed

@security @owasp-a03
Scenario: SQL injection in email field is sanitized
  Given the user is on the login page
  When the user enters "admin' OR '1'='1" in the email field
  Then the input is sanitized and login fails with "Invalid credentials"
```

### Portable Best Practices

- **Isolate instructions in the system message.** Place role and style rules in the system message to reduce instruction dilution.
- **Place criteria adjacent to the generation instruction.** Use delimiters (`---CRITERIA---`) and position them immediately before the task to leverage recency bias.
- **Enforce schema with explicit fields.** List required Gherkin components (Feature, Scenario, Given/When/Then, tags) and demand output-only blocks.
- **Set temperature ≤ 0.2.** Low temperature reduces variance and improves schema compliance.
- **Validate with a Gherkin parser.** Run outputs through a parser (e.g., `behave --dry-run`) and fail on syntax errors.

### Secure API Key Loading

This code block securely loads API keys from Colab userdata, ensuring keys are never hardcoded.

In [ ]:
import os
from google.colab import userdata
from google.colab.userdata import SecretNotFoundError

def load_api_keys(required_keys):
    """
    Securely loads API keys from Google Colab userdata and sets them as environment variables.

    Args:
        required_keys (list of str): List of environment variable names to load from Colab secrets.

    Returns:
        None

    Raises:
        EnvironmentError: If any required key is missing from Colab secrets.
    """
    missing = []
    for k in required_keys:
        value = None
        try:
            value = userdata.get(k)
        except SecretNotFoundError:
            pass

        os.environ[k] = value if value is not None else ""

        if not os.environ[k]:
            missing.append(k)

    if missing:
        raise EnvironmentError(
            f"Missing keys: {', '.join(missing)}. "
            "Add them in Colab → Settings → Secrets."
        )

    print("All keys loaded.")

REQUIRED_KEYS = ["OPENAI_API_KEY", "ANTHROPIC_API_KEY"]
load_api_keys(REQUIRED_KEYS)

## Key Takeaways

- **Use a five-part scaffold.** Role, context with delimiters, task with counts, format schema, and constraints.
- **Delimiters prevent format contamination.** Wrap acceptance criteria and output blocks with clear markers.
- **Explicit counts and tags drive coverage.** Specify how many scenarios per type and require tags like `@security` or `@owasp-a01`.
- **Low temperature improves determinism.** Set temperature ≤ 0.2 for schema compliance.
- **Validate outputs with a parser.** Fail fast on syntax errors to ensure runner compatibility.

### When to Use This Pattern

- Outputs drift to happy paths only.
- Models ignore acceptance criteria.
- You need schema-valid, parseable Gherkin for CI runners.
- Security or boundary cases are missing.